In [1]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from IPython.display import display, Markdown
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, BaseChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import gradio as gr

c:\Users\Bogdan Andrei Baltes\miniconda3\envs\viu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# get a token: https://platform.openai.com/account/api-keys

from getpass import getpass

OPENAI_API_KEY = getpass()

In [3]:
from prompts import prompts

instruct = ChatPromptTemplate.from_template(prompts.template_sin_definir_claim)

In [137]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI


def query_pdf(query):
    # Load document using PyPDFLoader document loader
    loader = DirectoryLoader("data/sources_txt/")
    documents = loader.load()
    # Split document in chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
    docs = text_splitter.split_documents(documents=documents)

    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
    # Create vectors
    vectorstore = FAISS.from_documents(docs, embeddings)
    # Persist the vectors locally on disk
    vectorstore.save_local("faiss_index_constitution")

    # Load from local storage
    persisted_vectorstore = FAISS.load_local("faiss_index_constitution", embeddings)

    # Use RetrievalQA chain for orchestration
    qa = RetrievalQA.from_chain_type(llm=OpenAI(api_key=OPENAI_API_KEY), chain_type="stuff", retriever=persisted_vectorstore.as_retriever())
    result = qa.run(query)
    print(result)


def main():
    query = input("Type in your query: \n")
    while query != "exit":
        query_pdf(query)
        query = input("Type in your query: \n")


if __name__ == "__main__":
    main()

c:\Users\Bogdan Andrei Baltes\miniconda3\envs\viu\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


 I do not understand what you are asking. Could you please provide a specific question related to the context provided?
 Yolanda Díaz is the current vicepresidenta segunda del Gobierno and ministra de Trabajo y Economía Social in Spain. She is from Galicia and has a daughter. In a recent interview, she mentioned that she enjoys doing squats and her daughter has criticized her for talking too much during political rallies. She also stated that living in Madrid, despite her spacious official residence, is like a "condena" for her because she misses the sea.
 I don't know.


KeyboardInterrupt: 

In [4]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnableParallel
from langchain_community.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain_core.runnables import RunnableParallel
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

# embeddings = HuggingFaceEmbeddings()
# text = "This is a test document."
# query_result = embeddings.embed_query(text)

# query_result[:3]

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

In [5]:
from langchain.text_splitter import CharacterTextSplitter


# Load, chunk and index the contents of the blog.
# loader = PyPDFDirectoryLoader("data/sources_english/")
# loader = DirectoryLoader("data/sources_txt/")
# pages = loader.load_and_split()

loader = DirectoryLoader("data/sources_txt/")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=30, separator=".")
docs = text_splitter.split_documents(documents=documents)

vectorstore = FAISS.from_documents(documents=docs, embedding=embeddings)

Created a chunk of size 455, which is longer than the specified 200
Created a chunk of size 212, which is longer than the specified 200
Created a chunk of size 260, which is longer than the specified 200
Created a chunk of size 240, which is longer than the specified 200
Created a chunk of size 221, which is longer than the specified 200
Created a chunk of size 243, which is longer than the specified 200
Created a chunk of size 260, which is longer than the specified 200
Created a chunk of size 215, which is longer than the specified 200
Created a chunk of size 242, which is longer than the specified 200
Created a chunk of size 240, which is longer than the specified 200
Created a chunk of size 221, which is longer than the specified 200
Created a chunk of size 243, which is longer than the specified 200
Created a chunk of size 218, which is longer than the specified 200
Created a chunk of size 407, which is longer than the specified 200
Created a chunk of size 203, which is longer tha

In [6]:
q = "Yolanda Díaz"
docs = vectorstore.similarity_search(q, 3)
docs

[Document(page_content='En un momento de la entrevista, Yolanda Díaz y Marc Giró jugaron a un juego de adivinanzas en base a dar datos a tu rival para acertar cuál era el personaje que se tenía que adivinar', metadata={'source': 'data\\sources_txt\\okdiario.txt'}),
 Document(page_content='Yolanda Díaz y Marc Giró se pusieron a practicar este ejercicio, aunque el presentador no logró hacerlo de forma satisfactoria', metadata={'source': 'data\\sources_txt\\okdiario.txt'}),
 Document(page_content='Como experiencia personal y familiar, Yolanda Díaz admitió que su hija le suele decir que se «enrolla» a veces «demasiado» en los mítines que suele hacer por toda España con Sumar', metadata={'source': 'data\\sources_txt\\okdiario.txt'})]

In [6]:
from langchain_community.llms import Anyscale
import os

ANYSCALE_API_BASE = "https://api.endpoints.anyscale.com/v1"

ANYSCALE_MODEL_NAME = "mistralai/Mixtral-8x7B-Instruct-v0.1"

os.environ["ANYSCALE_API_BASE"] = ANYSCALE_API_BASE
os.environ["ANYSCALE_API_KEY"] = ANYSCALE_API_KEY

llm = Anyscale(model_name=ANYSCALE_MODEL_NAME)

In [7]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = instruct # hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0, openai_api_key=OPENAI_API_KEY)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [8]:
response = rag_chain_with_source.invoke("El número de parados disminuye en España")
# display(response['context'])
Markdown(response['answer'])
# Markdown(response)

**Veredicto**: Verdadero

**Justificación**: La afirmación "El número de parados disminuye en España" se verifica como verdadera basándose en la información proporcionada. Se menciona específicamente que "el número de parados siguió a la baja" y se detalla que en 2022, la cantidad de desempleados bajó en 79.800 personas y este año ha caído el doble (-193.400 personas). Además, se añade que de 2013 a 2019, el paro descendió a un ritmo de 404.000 personas anuales, lo que confirma una tendencia a la baja en el número de parados en España. Por lo tanto, la afirmación se sostiene con los datos presentados.

**Fuentes**:
- No existe información en la base de contexto actual sobre esta afirmación.

In [145]:
response['context'][1].metadata.get('source')

'data\\sources_txt\\elmundo.txt'

In [175]:
response

{'context': [Document(page_content='000 su número de paradas.\n\nPor sectores, el mayor aumento del empleo se dio en los Servicios, con 629.200 ocupados más, pero también fue el único que engrosó su paro (4.100 más)', metadata={'source': 'data\\sources_txt\\elplural.txt'}),
  Document(page_content='000 personas, y es el mayor aumento que se produce en España en la ocupación desde 2005 si no se tiene en cuenta 2021 -en el que se recuperaron los empleos de 2020 y se crearon 840.000 puestos-', metadata={'source': 'data\\sources_txt\\elmundo.txt'}),
  Document(page_content='600 personas El total de ocupados se mantiene por encima de los 21,2 millones mientras el paro alcanza a 2,8 millones de españoles, situándose la tasa en el 11,7% El recorte de jornada laboral que planea Díaz liquidará un tercio de la creación de empleo para 2024\n\nEl mercado laboral español mantuvo el pulso de la creación de empleo a pesar el agotamiento que mostró la actividad económica en la segunda mitad del pasado

In [9]:
def ask_question(input, history):
    # answer = chain.invoke(input)
    response = rag_chain_with_source.invoke(input)
    # return answer
    return response['answer'], response['context']

gr.Interface(ask_question,
            inputs=gr.Textbox(label="Inserte la afirmación que desee verificar"),
             outputs=[gr.Markdown('textbox', label="Resultado"), gr.Textbox(label="Referencias")],
             title='Verificando información desde la prensa',
             theme=gr.themes.Soft()).launch()

c:\Users\Bogdan Andrei Baltes\miniconda3\envs\viu\lib\site-packages\gradio\utils.py:860: UserWarning: Expected 2 arguments for function <function ask_question at 0x000002A4CCA75790>, received 1.
  warnings.warn(
c:\Users\Bogdan Andrei Baltes\miniconda3\envs\viu\lib\site-packages\gradio\utils.py:864: UserWarning: Expected at least 2 arguments for function <function ask_question at 0x000002A4CCA75790>, received 1.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\Bogdan Andrei Baltes\miniconda3\envs\viu\lib\site-packages\gradio\helpers.py:837: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
c:\Users\Bogdan Andrei Baltes\miniconda3\envs\viu\lib\site-packages\gradio\helpers.py:837: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
c:\Users\Bogdan Andrei Baltes\miniconda3\envs\viu\lib\site-packages\gradio\helpers.py:837: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


In [15]:
from langchain.chains import RetrievalQA

qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

claim = template_english + "Employment decreased in 2023."
response = qa_stuff.run(claim)
display(Markdown(response))


c:\Users\Bogdan Andrei Baltes\miniconda3\envs\viu\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


**Verdict**: False

**Justification**: The information is false according to the context. The Active Population Survey (EPA) for the fourth quarter of 2023, published by the National Statistics Institute (INE), confirmed that 783,000 jobs were created in 2023, raising the number of Social Security affiliates to 21,246,900 people. The growth experienced by employment amounted to 3.8%, almost three times higher than at the end of 2022. This data is supported by the elplural.com article from 01/26/2024, which confirms the increase in employment and the fall in unemployment. Therefore, the claim that employment decreased in 2023 is false based on the provided context.

**Sources**:
- ABC: No information
- ElMundo: No information
- ElPlural: True
- OKDiario: No information